# openai_api walkthrough（细讲版）

## 目标与先修
- 目标：系统理解 OpenAI API 能力全景、关键参数、以及 DeepSeek 兼容实调结果。
- 先修：了解端点概念、JSON、HTTP 状态码基础。


## 流程总览
1. 查看 capability overview（能做什么）。
2. 查看 parameter reference（参数作用与全量索引）。
3. 调用 `responses/embeddings/moderations` 并观察 `status`。
4. 通过 `run_demo` 汇总兼容性结果。


### 环境检查（在线模块）
- 本步做什么：检查 `DEEPSEEK_API_KEY` 是否存在。
- 为什么这样做：`openai_api` 依赖在线 LLM API；缺 key 时必须显式失败，避免误以为流程成功。
- 输入：环境变量 `DEEPSEEK_API_KEY`。
- 输出：通过则继续执行；失败则抛出 `RuntimeError`。
- 观察点：失败信息要明确提示“这是配置问题，不是业务逻辑问题”。


In [ ]:
import os

api_key = os.getenv("DEEPSEEK_API_KEY", "")
if not api_key:
    raise RuntimeError("缺少 DEEPSEEK_API_KEY：在线 walkthrough 不能继续执行。")

print("环境检查通过：已检测到 DEEPSEEK_API_KEY")


### 工具单元（统一 helper）
- 本步做什么：定义 `show_json` 与 `show_kv`，用于稳定展示中间对象。
- 为什么这样做：教学时必须让过程可观测，避免“函数跑完但看不到内部结构”。
- 输入：任意 Python 对象。
- 输出：可读的 JSON 字符串或键值对列表。
- 观察点：后续每步都复用这两个 helper，输出格式保持一致。


In [ ]:
import json


def show_json(obj):
    print(json.dumps(obj, ensure_ascii=False, indent=2, default=str))


def show_kv(title, mapping):
    print(title)
    for key, value in mapping.items():
        print(f"- {key}: {value}")


## 步骤拆解（逐步）

### Step 1: 导入函数并准备输入
- 本步做什么：导入能力矩阵、参数索引、在线调用和 demo 函数。
- 为什么这样做：后续可分别观察“概念层”和“实调层”。
- 输入：用户文本 `user_text`。
- 输出：可复用的函数与输入变量。
- 观察点：保持同一输入，便于比较三个端点返回状态。


In [ ]:
from component import (
    build_capability_overview,
    build_parameter_reference,
    request_embeddings_demo,
    request_moderations_demo,
    request_responses_demo,
    run_demo,
)

user_text = "请用简短中文说明 LLM API 常见能力与参数作用。"
print(user_text)


### Step 2: 查看 capability overview
- 本步做什么：调用 `build_capability_overview()` 获取能力矩阵。
- 为什么这样做：先建立 API 能力边界，再看具体端点参数。
- 输入：无。
- 输出：能力列表（含 capability/api_surface/when_to_use）。
- 观察点：确认覆盖文本生成、工具调用、结构化输出、安全审核等类别。


In [ ]:
overview = build_capability_overview()
print(f"能力条目数量: {len(overview)}")
for idx, item in enumerate(overview, start=1):
    print(f"[{idx}] {item['capability']} -> {item['api_surface']}")

show_json(overview[0])


### Step 3: 查看 parameter reference
- 本步做什么：调用 `build_parameter_reference()`，查看核心参数与全量参数名。
- 为什么这样做：教学中需要“重点参数解释 + 全量索引”两层视角。
- 输入：无。
- 输出：按端点分组的参数参考字典。
- 观察点：关注 `required_params`、`core_params`、`all_param_names` 三块。


In [ ]:
reference = build_parameter_reference()
print("端点列表:", list(reference.keys()))

for endpoint, payload in reference.items():
    print()
    print(f"=== {endpoint} ===")
    print("required:", payload["required_params"])
    print("core keys:", sorted(payload["core_params"].keys()))
    print("all params count:", len(payload["all_param_names"]))

show_json(reference["responses"])


### Step 4: 逐个调用三个重点端点
- 本步做什么：分别调用 `request_responses_demo`、`request_embeddings_demo`、`request_moderations_demo`。
- 为什么这样做：不同供应商兼容度不同，需要结构化记录每个端点状态。
- 输入：同一 `user_text`。
- 输出：三个 `EndpointCallResult`。
- 观察点：重点看 `status/http_status/provider_error_type/summary`。


In [ ]:
responses_result = request_responses_demo(user_text)
embeddings_result = request_embeddings_demo(user_text)
moderations_result = request_moderations_demo(user_text)

print("responses:")
show_json(responses_result)
print()
print("embeddings:")
show_json(embeddings_result)
print()
print("moderations:")
show_json(moderations_result)


### Step 5: 解释 status 分类含义
- 本步做什么：把端点结果中的 `status` 统一汇总并解释。
- 为什么这样做：教学重点是“知道端点是否可用，以及为什么不可用”。
- 输入：三个端点调用结果。
- 输出：状态汇总表。
- 观察点：`ok` 表示可用；`unsupported_by_provider` 表示兼容差异；其他状态对应鉴权、限流、网络等问题。


In [ ]:
status_map = {
    "ok": "调用成功",
    "unsupported_by_provider": "供应商不支持该端点",
    "auth_error": "鉴权失败（key/权限问题）",
    "rate_limited": "触发限流",
    "provider_server_error": "服务端错误",
    "network_error": "网络或超时错误",
    "unknown_error": "未分类错误",
}

endpoint_results = {
    "responses": responses_result,
    "embeddings": embeddings_result,
    "moderations": moderations_result,
}

for endpoint, payload in endpoint_results.items():
    status = payload.get("status", "unknown_error")
    meaning = status_map.get(status, "未定义状态")
    print(f"- {endpoint}: {status} -> {meaning}")


## 端到端结果

### Step 6: 运行 run_demo 获取最终教学总结
- 本步做什么：执行 `run_demo(user_text)`。
- 为什么这样做：验证最终总结是否正确合并了能力概念与兼容实调结果。
- 输入：`user_text`。
- 输出：`final_answer` + `trace`。
- 观察点：最终文本中必须明确标注这是“OpenAI API 概念 + DeepSeek 兼容实调结果”。


In [ ]:
result = run_demo(user_text)
show_json(result["trace"])

print()
print("FINAL ANSWER:")
print(result["final_answer"])


## 常见错误
- `RuntimeError: 缺少 DEEPSEEK_API_KEY`：未配置在线 key。
- `status=unsupported_by_provider`：供应商兼容差异，不是代码 bug。
- `status=auth_error/rate_limited/network_error`：分别对应鉴权、限流、网络问题。

## 总结
- 学习 API 时要分两层：先看概念与参数，再看供应商真实可用性。
- `status` 分类是在线教学可观测性的核心指标。
